In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import pandas as pd
import os
# from pl_bolts.models.autoencoders import VAE
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
# os.getcwd()
root_dir = "/workspace/vae-fairness/"
data_dir = os.path.join(root_dir,"data")
raw_data_dir = os.path.join(data_dir,"raw")
raw_german_all = os.path.join(raw_data_dir,"german.data")
raw_bar_pass = os.path.join(raw_data_dir,"bar_pass_prediction.csv")
# raw_german_all

In [5]:
bar_df = pd.read_csv(raw_bar_pass).set_index('ID')
bar_df

,decile1b,decile3,decile1,sex,race,cluster,lsat,ugpa,zfygpa,DOB_yr,grad,zgpa,bar1,bar1_yr,bar2,bar2_yr,fulltime,fam_inc,age,gender,parttime,male,race1,race2,Dropout,other,asian,black,hisp,pass_bar,bar,bar_passed,tier,index6040,indxgrp,indxgrp2,dnn_bar_pass_prediction,gpa
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,10.0,10.0,10.0,1.0,7.0,1.0,44.0,3.5,1.33,69.0,Y,1.88,P,7.0,P,94.0,1.0,5.0,-62.0,female,0.0,0.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,4.0,886.842082,g 700+,i 820+,0.979804,3.5
3,5.0,4.0,5.0,1.0,7.0,2.0,29.0,3.5,-0.11,69.0,Y,-0.57,P,7.0,P,94.0,1.0,4.0,-62.0,female,0.0,0.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,2.0,649.999987,f 640-700,f 640-700,0.979804,3.5
36,3.0,2.0,3.0,2.0,7.0,3.0,36.0,3.5,-0.64,65.0,Y,-1.12,P,7.0,P,94.0,1.0,1.0,-58.0,male,0.0,1.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,3.0,760.526298,g 700+,h 760-820,0.979804,3.5
52,7.0,4.0,7.0,2.0,7.0,3.0,39.0,3.5,0.34,58.0,Y,-0.49,P,7.0,P,94.0,1.0,4.0,-51.0,male,0.0,1.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,3.0,807.894717,g 700+,h 760-820,0.979804,3.5
55,9.0,8.0,9.0,2.0,7.0,4.0,48.0,3.5,1.02,68.0,Y,0.76,P,7.0,P,94.0,1.0,4.0,-61.0,male,0.0,1.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,5.0,949.999974,g 700+,i 820+,0.979804,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10695,3.0,1.0,3.0,2.0,3.0,6.0,26.5,1.8,-0.58,64.0,Y,-1.49,F,2.0,F,95.0,1.0,2.0,-62.0,male,0.0,1.0,black,a black,NO,0,0,1,0,0,c Failed,False,1.0,440.526304,b 400-460,b 400-460,0.557568,1.8
19642,2.0,1.0,3.0,2.0,3.0,6.0,19.7,1.8,-0.64,64.0,Y,-1.36,F,7.0,F,94.0,1.0,3.0,-57.0,male,0.0,1.0,black,a black,NO,0,0,1,0,0,c Failed,False,1.0,333.157888,a under 400,a under 400,0.557568,1.8
24704,7.0,8.0,7.0,2.0,3.0,3.0,36.0,1.8,0.41,66.0,Y,0.80,P,7.0,P,95.0,2.0,3.0,-59.0,male,1.0,1.0,black,a black,NO,0,0,1,0,1,a Passed 1st time,True,3.0,590.526298,e 580-640,e 580-640,0.557568,1.8


In [6]:

bar_df['age'] = bar_df['age'].abs()

drop_cols_dup = ['DOB_yr','sex','bar2','race2','other','asian','black','hisp','bar','bar_passed','indxgrp','ugpa','indxgrp2','gender','race1'] #,'bar2_yr'
drop_cols_na = ['decile1b','decile3','decile1','zgpa','zfygpa']
one_hot_cols = ['grad','bar1','race','Dropout']
bar_df = bar_df.drop(columns = drop_cols_dup + drop_cols_na)
bar_df = bar_df.dropna()
bar_df = pd.get_dummies(bar_df,columns = one_hot_cols)
# sens_attri = ['other','asian','black','hisp', 'male','race1','race2', 'sex','race','DOB_yr','age','gender']

In [7]:
bar_df.shape

(21894, 29)

In [8]:
X_train, X_test = train_test_split(bar_df, test_size=0.2, random_state=42)
scaler = MinMaxScaler()#StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# torch.tensor(X_train_scaled.values,dtype=torch.float32)#.shape
X_train_torch = torch.tensor(X_train_scaled,dtype=torch.float32)#.shape
X_test_torch = torch.tensor(X_test_scaled,dtype=torch.float32)#.shape
# trainloader=DataLoader(dataset=X_train_torch,batch_size=1024)
# testloader=DataLoader(dataset=X_test_torch,batch_size=1024)

trainloader = DataLoader(dataset=X_train_torch,batch_size=1024)
testloader = DataLoader(dataset=X_test_torch,batch_size=1024)


In [16]:
trainloader.dataset.shape

torch.Size([17515, 29])

In [19]:
class Autoencoder(nn.Module):
    def __init__(self,n_feats,layer_shape = {1:15,2:7,3:7}):
        super(Autoencoder,self).__init__()
        self.encoder = torch.nn.Sequential(self._generate_layers(n_feats = n_feats,layer_shape = layer_shape,block = 'encoder'))
        self.decoder = torch.nn.Sequential(self._generate_layers(n_feats = n_feats,layer_shape = layer_shape,block = 'decoder'))
        
    def _generate_layers(self,n_feats,layer_shape,block):
        input_layer_shape = {0:n_feats}
        input_layer_shape.update(layer_shape)
        layer_shape = input_layer_shape
        if block == 'decoder':
            layer_shape = dict(reversed(list(layer_shape.items())))
        layers = []
#         first_layer = next(iter(layer_shape))
#         layers.append(nn.Linear(layer_shape[first_layer],layer_shape[first_layer + 1]))
#         layers.append(nn.BatchNorm1d(num_features = layer_shape[first_layer+1]))
#         layers.append(nn.ReLU())
        for index,layer in enumerate(layer_shape):
            if index == len(layer_shape) - 1:
                # skipping last layer 
                break
            if block == 'decoder':
                next_layer = layer - 1
            elif block == 'encoder':
                next_layer = layer + 1
            layers.append(nn.Linear(layer_shape[layer],layer_shape[next_layer]))
            layers.append(nn.BatchNorm1d(num_features = layer_shape[next_layer]))
            layers.append(nn.ReLU())
        return nn.Sequential(*layers)
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
n_feats = X_train_torch.shape[1]

model = Autoencoder(n_feats).to(device)

# summary(model, (15, 29))
print(model)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.MSELoss()
 
# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(model.parameters(),
                             lr = 1e-3,
                             weight_decay = 1e-8)

for epoch in range(100):
    for idx, x in enumerate(trainloader):
#         x = x.squeeze()
#         x = x / x.max()
        x_pred = model(x) # forward pass
        loss = loss_fn(x_pred, x)
        if idx % 1024 == 0:
            print(epoch, loss.item())
        optimizer.zero_grad()
        loss.backward()         # backward pass
        optimizer.step()

Autoencoder(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=29, out_features=15, bias=True)
      (1): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=15, out_features=7, bias=True)
      (4): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Linear(in_features=7, out_features=7, bias=True)
      (7): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
    )
  )
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=7, out_features=7, bias=True)
      (1): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=7, out_features=15, bias=True)
      (4): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Linear(in_features=15, out_f

In [22]:
def loss_function(recon_x, x, mu, logvar):
    # https://arxiv.org/abs/1312.6114 (Appendix B)
    cross_entropy = F.binary_cross_entropy(recon_x, x, size_average=False)        
    kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return cross_entropy + kl

In [24]:
class VAE(nn.Module):
    def __init__(self,n_feats,layer_shape = {1:15,2:7,3:7}):
        super(VAE,self).__init__()
        self.encoder = torch.nn.Sequential(self._generate_layers(n_feats = n_feats,layer_shape = layer_shape,block = 'encoder'))
        self.decoder = torch.nn.Sequential(self._generate_layers(n_feats = n_feats,layer_shape = layer_shape,block = 'decoder'))
        latent_dim = layer_shape[max(layer_shape, key=int)]
        self.mu_layer = nn.Linear(latent_dim, latent_dim)  # mu layer
        self.logvar_layer = nn.Linear(latent_dim, latent_dim)  # logvariance layer
        
        
        
    def _generate_layers(self,n_feats,layer_shape,block):
        input_layer_shape = {0:n_feats}
        input_layer_shape.update(layer_shape)
        layer_shape = input_layer_shape
        if block == 'decoder':
            layer_shape = dict(reversed(list(layer_shape.items())))
        layers = []
#         first_layer = next(iter(layer_shape))
#         layers.append(nn.Linear(layer_shape[first_layer],layer_shape[first_layer + 1]))
#         layers.append(nn.BatchNorm1d(num_features = layer_shape[first_layer+1]))
#         layers.append(nn.ReLU())
        for index,layer in enumerate(layer_shape):
            if index == len(layer_shape) - 1:
                # skipping last layer 
                break
            if block == 'decoder':
                next_layer = layer - 1
            elif block == 'encoder':
                next_layer = layer + 1
            layers.append(nn.Linear(layer_shape[layer],layer_shape[next_layer]))
            layers.append(nn.BatchNorm1d(num_features = layer_shape[next_layer]))
            layers.append(nn.ReLU())
        return nn.Sequential(*layers)
    
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling
        return sample
    
    def forward(self, x):
        hidden = self.encoder(x)
        # get `mu` and `log_var`
        mu = self.mu_layer(hidden)
        logvar = self.logvar_layer(hidden)
        z = self.reparameterize(mu, logvar)
#         z = self.fc2(z)
        decoded = self.decoder(z)
        return decoded, mu, logvar
    
    
    
n_feats = X_train_torch.shape[1]

model = VAE(n_feats).to(device)

# summary(model, (15, 29))
print(model)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.MSELoss()
 
# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(model.parameters(),
                             lr = 1e-3,
                             weight_decay = 1e-8)

for epoch in range(100):
    for idx, x in enumerate(trainloader):
#         x = x.squeeze()
#         x = x / x.max()
        recon_x, mu, logvar = model(x) # forward pass
        loss = loss_function(recon_x, x, mu, logvar)
#         loss = loss_fn(x_pred, x)
        if idx % 1024 == 0:
            print(epoch, loss.item())
        optimizer.zero_grad()
        loss.backward()         # backward pass
        optimizer.step()

VAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=29, out_features=15, bias=True)
      (1): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=15, out_features=7, bias=True)
      (4): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Linear(in_features=7, out_features=7, bias=True)
      (7): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
    )
  )
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=7, out_features=7, bias=True)
      (1): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=7, out_features=15, bias=True)
      (4): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Linear(in_features=15, out_features=

/opt/conda/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


RuntimeError: all elements of input should be between 0 and 1

In [ ]:
class Autoencoder(nn.Module):
    def __init__(n_feats,layer_shape = {1:15,2:7,3:7}):
        super(self,Autoencoder).__init__()
        self.encoder = torch.nn.Sequential(self._generate_layers(n_feats,layer_shape,block = 'encoder'))
        self.decoder = torch.nn.Sequential(self._generate_layers(n_feats,layer_shape,block = 'decoder'))
        
    def _generate_layers(n_feats,layer_shape,block):
        input_layer_shape = {0:n_feats}
        input_layer_shape.update(layer_shape)
        layer_shape = input_layer_shape
        if block == 'decoder':
            layer_shape = dict(reversed(list(layer_shape.items())))
        layers = []
#         first_layer = next(iter(layer_shape))
#         layers.append(nn.Linear(layer_shape[first_layer],layer_shape[first_layer + 1]))
#         layers.append(nn.BatchNorm1d(num_features = layer_shape[first_layer+1]))
#         layers.append(nn.ReLU())
        for index,layer in enumerate(layer_shape):
            if index == len(layer_shape) - 1:
                break
            if block == 'decoder':
                next_layer = layer - 1
            elif block == 'encoder':
                next_layer = layer + 1
            layers.append(nn.Linear(layer_shape[layer],layer_shape[next_layer]))
            layers.append(nn.BatchNorm1d(num_features = layer_shape[next_layer]))
            layers.append(nn.ReLU())
        return nn.Sequential(*layers)
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
n_feats = X_train_torch.shape[1]

model = AutoEncoder(n_feats).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

loss_mse = nn.MSELoss()


epochs = 1500
log_interval = 50
val_losses = []
train_losses = []
test_losses = []

def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(trainloader):
        print(data)
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    if epoch % 200 == 0:        
        print('====> Epoch: {} Average training loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        train_losses.append(train_loss / len(trainloader.dataset))
        
def test(epoch):
    with torch.no_grad():
        test_loss = 0
        for batch_idx, data in enumerate(testloader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            loss = loss_mse(recon_batch, data, mu, logvar)
            test_loss += loss.item()
            if epoch % 200 == 0:        
                print('====> Epoch: {} Average test loss: {:.4f}'.format(
                    epoch, test_loss / len(testloader.dataset)))
            test_losses.append(test_loss / len(testloader.dataset))